In [ ]:
import mne
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from metadata import *

print(subject_list)
print(tmin, tmax)

In [ ]:
picks = ['A31'] # or try: ['A1', 'A2', 'A4', 'A5', 'A26', 'A27', 'A29', 'A30', 'A31', 'A32']

In [ ]:
# FOR EACH SUBJECT, GRAND AVERAGE FOR EACH CONDITION

evoked_dict = {}

for subject in subject_list: 
    print('##### Processing subject', subject, '...')

    print('# Reading raw data...')
    raw = mne.io.read_raw_fif(
        ica_after_files[subject], 
        preload = True
    )
    events = mne.find_events(raw)

    print('# Epoching...')
    epochs = mne.Epochs(
        raw, 
        events = events, 
        event_id = event_id, 
        tmin = tmin, 
        tmax = tmax, 
        baseline = (tmin, 0)
    )
    evoked_dict["standard/can"] = epochs['standard/can'].average()
    evoked_dict["standard/rev"] = epochs['standard/rev'].average()
    evoked_dict["deviant/can"] = epochs['deviant/can'].average()
    evoked_dict["deviant/rev"] = epochs['deviant/rev'].average()

    print('# Visualizing subject', subject, '...')
    figure = mne.viz.plot_compare_evokeds(
        evoked_dict, 
        picks = picks, 
        show_sensors = False,
        colors = colors,
        linestyles = linestyles,
        title = 'Subject_'+subject
    )

    print('# Saving subject', subject, '...')
    figure[0].savefig(viz_files[subject])

print('***** ALL DONE *****')

In [ ]:
# ACROSS SUBJECTS, GRAND AVERAGE FOR EACH CONDITION

title = 'AverageFZ'
evoked_dict = {}

for condition in conditions: 
    print('##### Processing condition', condition, '...')
    evoked = []
    for subject in subject_list: 
        print('##### Processing subject', subject, '...')

        print('# Reading raw data...')
        raw = mne.io.read_raw_fif(
            ica_after_files[subject], 
            preload = True
        )
        events = mne.find_events(raw)

        print('# Epoching...')
        epochs = mne.Epochs(
            raw, 
            events = events, 
            event_id = event_id, 
            tmin = tmin, 
            tmax = tmax, 
            baseline = (tmin, 0)
        )

        print('# Processing data...')
        average = epochs[condition].average()
        evoked.append(average)
    sd_block = mne.grand_average(evoked)
    evoked_dict[condition] = sd_block

figure = mne.viz.plot_compare_evokeds(
    evoked_dict, 
    picks = picks,
    show_sensors = False,
    colors = colors,
    linestyles = linestyles,
    title = title
)

figure[0].savefig(vizdir+title)

print('***** ALL DONE *****')

In [ ]:
# ACROSS SUBJECTS, DIFFERENCE BETWEEN CAN/REV

title = 'AverageFZ_DifferenceWave'
evoked_block = {}

for block in blocks:
    print('##### Processing block', block, '...')
    evoked = []
    for subject in subject_list: 
        print('##### Processing subject', subject, '...')

        print('# Reading raw data...')
        raw = mne.io.read_raw_fif(ica_after_files[subject], preload=True)
        events = mne.find_events(raw)

        print('# Epoching...')
        epochs = mne.Epochs(
            raw, 
            events = events, 
            event_id = event_id, 
            tmin = tmin, 
            tmax = tmax, 
            baseline = (tmin, 0)
        )

        print('# Processing data...')
        evoked_s = epochs['standard/'+block].average()
        evoked_d = epochs['deviant/'+block].average()
        diff = mne.combine_evoked([evoked_s, -evoked_d], weights='nave')
        diff.data = evoked_s.data-evoked_d.data
        evoked.append(diff)
    sd_block = mne.grand_average(evoked)
    evoked_block[block] = sd_block

figure = mne.viz.plot_compare_evokeds(
    evoked_block, 
    picks = picks,
    show_sensors = False,
    colors = colors,
    title = title
)

figure[0].savefig(vizdir+title)

print('***** ALL DONE *****')